In [13]:
import scipy.io as scio
import numpy as np
import tensorflow as tf
import keras
# from sklearn import svm
from keras.layers import Input, Dense, ZeroPadding2D, Dropout, Activation
from keras.layers import Input, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model
from keras import regularizers
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers
import time
import warnings
import os
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import layer_utils
from keras.utils.vis_utils import plot_model
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot
from numba import cuda
# from sklearn import datasets
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.multiclass import OneVsRestClassifier

os.environ["CUDA_VISIBLE_DEVICES"]="0"

%run Dataset.ipynb
# inputShape = (10,10,9)
inputShape = (37,5,9)
Ratio="R4peopleDataset"  #R4 or R1C
Pnumber=15
DsetName=str(Pnumber) + Ratio
data_train,data_test,label_trian,label_test = GetNew(inputShape,DsetName)
# data_train = data_train.reshape(np.shape(data_train)[0],100,1,9)
# data_test = data_test.reshape(np.shape(data_test)[0],100,1,9)
print("data loaded")

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()


def conv2d_bn(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same'):
    x = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(inpt)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x


def basic_bottle(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False):
    x = conv2d_bn(inpt, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters)
#     x = layers.Dropout(0.1)
    x = layers.BatchNormalization()(x)
    if if_baisc == True:
        temp = conv2d_bn(inpt, filters=filters, kernel_size=(1, 1), strides=2, padding='same')
        outt = layers.add([x, temp])
    else:
        outt = layers.add([x, inpt])
    return outt


def EMGNet_block(inpt, filters=64, kernel_size=(3, 3), strides=1, padding='same'):
    x = layers.Conv2D(filters=filters, kernel_size=(6, 2), strides=strides, padding='valid')(inpt)
    x = layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x_shortcut = x
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size)
    x = layers.add([x, x_shortcut])
    x_shortcut = x
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)
    x = conv2d_bn(x, filters=filters, kernel_size=kernel_size)
    x = layers.add([x, x_shortcut])
    return x


def CNN(input_shape=inputShape, classes=10):
    X_input = Input(input_shape)

    X = conv2d_bn(X_input, filters=64, kernel_size=(4, 4), strides=1, padding='valid') # 7 7 64

    # layer 2
    X = basic_bottle(X, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)#7 7 64
    X = basic_bottle(X, filters=64, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)#7 7 64
    X = layers.Dropout(.2)(X)
    # layer 3
    X = basic_bottle(X, filters=128, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True) #
    X = basic_bottle(X, filters=128, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # layer 4
    X = basic_bottle(X, filters=256, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    X = basic_bottle(X, filters=256, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # layer 5
    X = basic_bottle(X, filters=512, kernel_size=(3, 3), strides=2, padding='same', if_baisc=True)
    X = basic_bottle(X, filters=512, kernel_size=(3, 3), strides=1, padding='same', if_baisc=False)
    X = layers.Dropout(.2)(X)
    # GlobalAveragePool
    X = layers.GlobalAveragePooling2D()(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet18')
    return model

def EMGNet(input_shape=inputShape, classes=10):
    X_input = Input(input_shape)

    X = EMGNet_block(X_input, filters=64, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=128, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=256, kernel_size=(4, 2), strides=1, padding='same')
    X = EMGNet_block(X, filters=512, kernel_size=(4, 2), strides=1, padding='same')
    X = layers.GlobalAveragePooling2D()(X)
    X = layers.Dropout(.5)(X)
    X = layers.Dense(classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='EMGNet')
    return model



data loaded


In [14]:
# model = svm.SVC(gamma=1, kernel='rbf', C=0.1,  verbose=True, shrinking=True, max_iter=1, coef0=0.0, decision_function_shape='ovo')
model = EMGNet(input_shape=inputShape, classes=10)
model.summary()
# print(np.shape(data_train))
# svc = SVC()
# o_vs_r = OneVsRestClassifier(svc)
# o_vs_r.fit(data_train, label_trian)


Model: "EMGNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 37, 5, 9)]   0                                            
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 32, 4, 64)    6976        input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 32, 4, 64)    256         conv2d_31[0][0]                  
__________________________________________________________________________________________________
activation_31 (Activation)      (None, 32, 4, 64)    0           batch_normalization_39[0][0]     
_____________________________________________________________________________________________

In [16]:
import time
start = time.time()

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01/250, amsgrad=False), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), loss=categorical_squared_hinge, metrics=['accuracy'])
history = LossHistory() # 创建一个history实例

model.fit(data_train, label_trian, epochs=250, batch_size=128, verbose=1,
            validation_data=(data_test, label_test),callbacks=[history])
# pred_test = o_vs_r.predict(data_test.reshape(-1, 1))
# pred_test = o_vs_r.predict(data_test)
# pred_ytest = clf.predict((y_test).reshape(1,-1))
# model.save('TestANN.h5')
# print("test Accuracy:")
# print(f"{accuracy_score(label_test,pred_test ):.2%}\n")
preds_train = model.evaluate(data_train, label_trian)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(data_test, label_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))

end = time.time()
print("time:",end-start)

history.loss_plot('epoch')
cuda.select_device(0)
cuda.close()

Epoch 1/250
85/85 [==============================] - 17s 182ms/step - loss: 1.6928 - accuracy: 0.3915 - val_loss: 21375.4062 - val_accuracy: 0.1000
Epoch 2/250
85/85 [==============================] - 16s 183ms/step - loss: 1.0389 - accuracy: 0.5743 - val_loss: 365.9483 - val_accuracy: 0.2075
Epoch 3/250
85/85 [==============================] - 16s 184ms/step - loss: 0.8760 - accuracy: 0.6567 - val_loss: 48.8941 - val_accuracy: 0.3267
Epoch 4/250
85/85 [==============================] - 16s 183ms/step - loss: 0.7457 - accuracy: 0.7121 - val_loss: 2.2766 - val_accuracy: 0.5502
Epoch 5/250
85/85 [==============================] - 16s 183ms/step - loss: 0.6985 - accuracy: 0.7348 - val_loss: 4.8774 - val_accuracy: 0.5608
Epoch 6/250
85/85 [==============================] - 16s 183ms/step - loss: 0.7089 - accuracy: 0.7254 - val_loss: 2.2063 - val_accuracy: 0.5231
Epoch 7/250
85/85 [==============================] - 16s 184ms/step - loss: 0.6016 - accuracy: 0.7679 - val_loss: 0.8878 - val_ac

Epoch 56/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1634 - accuracy: 0.9400 - val_loss: 0.4652 - val_accuracy: 0.8980
Epoch 57/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1437 - accuracy: 0.9461 - val_loss: 0.3905 - val_accuracy: 0.9106
Epoch 58/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1543 - accuracy: 0.9463 - val_loss: 0.7507 - val_accuracy: 0.8584
Epoch 59/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1497 - accuracy: 0.9480 - val_loss: 0.4800 - val_accuracy: 0.8918
Epoch 60/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1452 - accuracy: 0.9476 - val_loss: 0.5069 - val_accuracy: 0.9000
Epoch 61/250
85/85 [==============================] - 15s 172ms/step - loss: 0.1362 - accuracy: 0.9508 - val_loss: 0.6528 - val_accuracy: 0.8663
Epoch 62/250
85/85 [==============================] - 16s 184ms/step - loss: 0.1374 - accuracy: 0.9506 - val_loss: 0.3828 - val_ac

85/85 [==============================] - 16s 184ms/step - loss: 0.0474 - accuracy: 0.9834 - val_loss: 0.6084 - val_accuracy: 0.9122
Epoch 113/250
85/85 [==============================] - 16s 184ms/step - loss: 0.0463 - accuracy: 0.9840 - val_loss: 0.5581 - val_accuracy: 0.9192
Epoch 114/250
85/85 [==============================] - 16s 184ms/step - loss: 0.0386 - accuracy: 0.9858 - val_loss: 0.5490 - val_accuracy: 0.9086
Epoch 115/250
85/85 [==============================] - 16s 184ms/step - loss: 0.0454 - accuracy: 0.9843 - val_loss: 0.6531 - val_accuracy: 0.8941
Epoch 116/250
85/85 [==============================] - 16s 184ms/step - loss: 0.0401 - accuracy: 0.9855 - val_loss: 0.5479 - val_accuracy: 0.9212
Epoch 117/250
85/85 [==============================] - 16s 185ms/step - loss: 0.0558 - accuracy: 0.9844 - val_loss: 0.5099 - val_accuracy: 0.9196
Epoch 118/250
85/85 [==============================] - 15s 176ms/step - loss: 0.1894 - accuracy: 0.9451 - val_loss: 0.5972 - val_accuracy:

KeyboardInterrupt: 